In [41]:
import os, sys
import nltk
import urllib.request, urllib.error
import requests
from urllib.parse import urljoin
from bs4 import *
import datetime
import io
import re

In [45]:
def savePage(url, pagefilename='page'):
    def soupfindnSave(pagefolder, tag2find='img', inner='src'):
        """saves on specified `pagefolder` all tag2find objects"""
        if not os.path.exists(pagefolder): # create only once
            os.mkdir(pagefolder)
        for res in soup.findAll(tag2find):   # images, css, etc..
            try:         
                if not res.has_attr(inner): # check if inner tag (file object) exists
                    continue # may or may not exist
                filename = re.sub('\W+', '', os.path.basename(res[inner])) # clean special chars
                fileurl = urljoin(url, res.get(inner))
                filepath = os.path.join(pagefolder, filename)
                # rename html ref so can move html and folder of files anywhere
                res[inner] = os.path.join(os.path.basename(pagefolder), filename)
                if not os.path.isfile(filepath): # was not downloaded
                    with open(filepath, 'wb') as file:
                        filebin = session.get(fileurl)
                        file.write(filebin.content)
            except Exception as exc:
                print(exc, file=sys.stderr)
        return soup
    
    session = requests.Session()
    #... whatever other requests config you need here
    response = session.get(url)
    soup = BeautifulSoup(response.text, features="lxml")
    pagefolder = pagefilename
    soup = soupfindnSave(pagefolder, 'img', 'src')
    soup = soupfindnSave(pagefolder, 'link', 'href')
    soup = soupfindnSave(pagefolder, 'script', 'src')
    with open(pagefilename+'.html', 'wb') as file:
        file.write(soup.prettify('utf-8'))
    return soup

soup = savePage('https://en.wikipedia.org/wiki/Special:Random', 'wiki')

[Errno 2] No such file or directory: 'wiki\\'
No connection adapters were found for 'mw-data:TemplateStyles:r971394783'
No connection adapters were found for 'mw-data:TemplateStyles:r951705291'


In [37]:
url = 'https://en.wikipedia.org/wiki/Special:Random'

response = urllib.request.urlopen(url)
webContent = response.read()

# print(webContent)
soup = BeautifulSoup(webContent, 'html.parser')



css_files = []

for css in soup.find_all("link"):
    if css.attrs.get("href"):
        # if the link tag has the 'href' attribute
        css_url = urljoin(url, css.attrs.get("href"))
        css_files.append(css_url)


print("Total CSS files in the page:", len(css_files))

with open("css_files.txt", "w") as f:
    for css_file in css_files:
        print(css_file, file=f)
        
        
# print('\n'.join([str(i) for i in soup.find_all('p')]))
# print([i for i in soup.find_all('p')])

with io.open("output1.html", "w", encoding="utf-8") as file:
    file.write(str(soup.title) + '\n'.join([str(i) for i in soup.find_all('p')]))

Total CSS files in the page: 18


In [3]:
def crawl(pages, depth=None):
    indexed_url = [] # a list for the main and sub-HTML websites in the main website
    for i in range(depth):
        for page in pages:
            if page not in indexed_url:
                indexed_url.append(page)
                try:
                    c = urllib.request.urlopen(page)
                except:
                    print( "Could not open %s" % page)
                    continue
                soup = BeautifulSoup(c.read())
                links = soup('a') #finding all the sub_links
                for link in links:
                    if 'href' in dict(link.attrs):
                        url = urllib.parse.urljoin(page, link['href'])
                        if url.find("'") != -1:
                                continue
                        url = url.split('#')[0] 
                        if url[0:4] == 'http':
                                indexed_url.append(url)
        pages = indexed_url
    return indexed_url


In [4]:
pagelist=["https://en.wikipedia.org/wiki/Python_%28programming_language%29"]
urls = crawl(pagelist, depth=1)
# print( urls )